In [58]:
from module.util import *
from src.class_BertForClassification_Pipeline import Pipeline as PL
from src.article_predict import *
from transformers import BertTokenizer, BertConfig
sentence_model_path = './ckpt/version_0.0/four_class_augmented_0526_v2.pkl'
article_model_path = './ckpt/article/0707_300limit_6epoch_binary.pt'
pretrained_model_name = 'hfl/chinese-bert-wwm-ext'
label_column_list = ['無標註','自殺與憂鬱', '自殺行為', '其他類型']
article_label_list = ['無危機文章', '高危機文章']
seed = 1234
# sample_article = "當憂鬱情緒低沉時，生活就像一片無光之地。一種無形的重擔，壓迫著你的心靈，讓你感到喘不過氣來。每一個簡單的日常任務，都變得困難重重，仿佛要走過漫長的沙漠。喜悅和快樂的感覺變得模糊，只留下空虛與無力感。然而，憂鬱並不是生命的終點。尋求幫助，並不代表你是軟弱的。透過適當的治療和支持，這個黑暗的隧道終究有盡頭。憂鬱讓你在生活中迷失方向，但請記住，即使在最黑暗的夜晚，也總會有繁星點綴，為你照亮前行的路。"
sample_article = "快樂是生活的閃爍星光，淺淺的微笑，和愛人共享的一頓晚餐，是好友的笑聲，是寵物的搖尾巴，是風中的舞蹈，是雨後的彩虹。它總是在不經意的瞬間出現，讓我們想起，無論生活多麼艱難，都有值得我們微笑和珍惜的美好。所以，讓我們捕捉每一個快樂的瞬間，並用心去享受它帶給我們的溫暖與喜悅。"
sample_sentences = [sent for sent in split_sentence(sample_article) if sent not in ['', ' ', '。', '，', '\n']]
paragraphs = [label+'的段落:' for label in label_column_list]

In [59]:
pl = PL(pretrained_model_name=pretrained_model_name, load_model_path=sentence_model_path, \
        num_labels = len(label_column_list), seed = seed)

pl.initialize_training()
for sample_sentence in sample_sentences:
    predict = pl.get_predict(pl.model, sample_sentence)
    paragraphs[predict] += ' ' + sample_sentence
    print(f'>>>>>Predicted result for "{sample_sentence}": {[label_column_list[predict]]}')
paragraphs = np.array([paragraphs])


There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 3090


Some weights of the model checkpoint at hfl/chinese-bert-wwm-ext were not used when initializing BertForClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForClassification were not initialized from the model checkpoint at hfl/chinese-bert-wwm-ext

>>>>>Load model weight from pretrained model path.
>>>>>Predicted result for "快樂是生活的閃爍星光": ['無標註']
>>>>>Predicted result for "淺淺的微笑": ['無標註']
>>>>>Predicted result for "和愛人共享的一頓晚餐": ['無標註']
>>>>>Predicted result for "是好友的笑聲": ['無標註']
>>>>>Predicted result for "是寵物的搖尾巴": ['無標註']
>>>>>Predicted result for "是風中的舞蹈": ['無標註']
>>>>>Predicted result for "是雨後的彩虹": ['無標註']
>>>>>Predicted result for "它總是在不經意的瞬間出現": ['無標註']
>>>>>Predicted result for "讓我們想起": ['無標註']
>>>>>Predicted result for "無論生活多麼艱難": ['無標註']
>>>>>Predicted result for "都有值得我們微笑和珍惜的美好": ['無標註']
>>>>>Predicted result for "所以": ['無標註']
>>>>>Predicted result for "讓我們捕捉每一個快樂的瞬間": ['無標註']
>>>>>Predicted result for "並用心去享受它帶給我們的溫暖與喜悅": ['無標註']


In [60]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = BertTokenizer.from_pretrained(pretrained_model_name)
config = BertConfig.from_pretrained(pretrained_model_name)
config.max_length = 512
article_model = MyModel(config, num_labels=2).to(device)
article_model.load_state_dict(torch.load(article_model_path))
article_model.eval()
test_dataset = TextClassificationDataset(paragraphs, [[0.0, 0.0]], tokenizer, 'test')

# optimizer = AdamW(model.parameters(), lr=lr)

In [64]:
y_pred = []
tokens_tensors, segments_tensors, masks_tensors,labels  = [t.to(device) for t in test_dataset[0]]
tokens_tensors = tokens_tensors.reshape(1, tokens_tensors.shape[0], tokens_tensors.shape[1])
segments_tensors = segments_tensors.reshape(1, segments_tensors.shape[0], segments_tensors.shape[1])
masks_tensors = masks_tensors.reshape(1, masks_tensors.shape[0], masks_tensors.shape[1])
labels = labels.reshape(1, labels.shape[0])
with torch.no_grad(): 
    loss, logits = article_model(input_ids=tokens_tensors, token_type_ids=segments_tensors, attention_mask=masks_tensors,labels=labels)
    prob = logits.data
    print(prob)
    _, pred = torch.max(prob, 1)
    print(pred)
print(paragraphs)
print(article_label_list[pred])

tensor([[0.0924, 0.9076]], device='cuda:0')
tensor([1], device='cuda:0')
[['無標註的段落: 快樂是生活的閃爍星光 淺淺的微笑 和愛人共享的一頓晚餐 是好友的笑聲 是寵物的搖尾巴 是風中的舞蹈 是雨後的彩虹 它總是在不經意的瞬間出現 讓我們想起 無論生活多麼艱難 都有值得我們微笑和珍惜的美好 所以 讓我們捕捉每一個快樂的瞬間 並用心去享受它帶給我們的溫暖與喜悅'
  '自殺與憂鬱的段落:' '自殺行為的段落:' '其他類型的段落:']]
高危機文章
